In [1]:
import sys
sys.path.append('/Users/zoltan/Documents/Rivabar/rivabar')

In [2]:
## Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rasterio
import pyproj
from rasterio.crs import CRS
import cartopy.crs as ccrs
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.ticker as mticker
from pystac_client import Client
import rivabar as rb

%matplotlib qt
plt.rcParams['svg.fonttype'] = 'none'

In [42]:
# Remove all rivabar modules from cache
import sys
modules_to_remove = [key for key in sys.modules.keys() if key.startswith('rivabar')]
for module in modules_to_remove:
    del sys.modules[module]

# Fresh import
import rivabar as rb

## Working with geemap to display and download mndwi images

### Create geemap map object

In [4]:
import ee
import geemap

In [5]:
ee.Authenticate()

True

In [6]:
# Initialize Earth Engine
ee.Initialize()

# Create a geemap
Map = geemap.Map()

# Load the official USGS WRS-2 grid
# This is the most reliable source
wrs2_grid = ee.FeatureCollection("projects/google/wrs2_descending")

# Alternative official dataset:
# wrs2_grid = ee.FeatureCollection("LANDSAT/WRS2_DESCENDING")

# Style the grid
grid_style = {
    'color': 'blue',
    'width': 2,
    'fillColor': '0000ff11'  # Very light blue fill with transparency
}

# Add to map with labels
Map.addLayer(wrs2_grid, grid_style, 'Landsat WRS-2 Path-Row Grid')

# Center on your area of interest
Map.setCenter(-60, -10, 6)

In [7]:
Map

Map(center=[-10, -60], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [1371]:
river_points = rb.collect_river_endpoints(Map)

Map(bottom=8949.0, center=[-10, -60], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=S…

In [1372]:
Map.add_marker(river_points['start'])
Map.add_marker(river_points['end'])

In [1373]:
river_points

{'start': [-16.957877, -64.719916], 'end': [-14.913846, -65.011012]}

In [8]:
river_points = {'start': [-13.972296, -67.531973], 'end': [-12.136923, -66.877997]} # Beni
# river_points = {'start': [-16.957877, -64.719916], 'end': [-14.913846, -65.011012]} # Mamore 1

In [9]:
import geopandas
wrs2_gdf = geopandas.read_file('/Users/zoltan/Documents/Rivabar/WRS2_descending_0/WRS2_descending.shp')
wrs2_gdf.head()

,AREA,PERIMETER,PR_,PR_ID,RINGS_OK,RINGS_NOK,PATH,ROW,MODE,SEQUENCE,WRSPR,PR,ACQDayL7,ACQDayL8,geometry
0,15.74326,26.98611,1.0,1.0,1,0,13,1,D,2233,013001,013001,1,9,"POLYGON ((-10.80341 80.9888, -8.97407 80.342, ..."
1,14.55366,25.84254,2.0,2.0,1,0,13,2,D,2234,013002,013002,1,9,"POLYGON ((-29.2425 80.18681, -29.29593 80.1989..."
2,13.37247,24.20303,3.0,3.0,1,0,13,3,D,2235,013003,013003,1,9,"POLYGON ((-24.04206 79.12261, -23.78294 79.063..."
3,12.26691,22.40265,4.0,4.0,1,0,13,4,D,2236,013004,013004,1,9,"POLYGON ((-36.66813 77.46094, -40.05219 78.098..."
4,11.26511,20.64284,5.0,5.0,1,0,13,5,D,2237,013005,013005,1,9,"POLYGON ((-44.1121 76.93656, -44.1247 76.9388,..."


## Beni

In [10]:
import ee
from shapely.geometry import Point

# Calculate midpoint of the two endpoints from river_points
start_lat, start_lon = river_points['start']
end_lat, end_lon = river_points['end']

# Calculate midpoint coordinates
midpoint_lon = (start_lon + end_lon) / 2
midpoint_lat = (start_lat + end_lat) / 2

print(f"Start point: [{start_lon}, {start_lat}]")
print(f"End point: [{end_lon}, {end_lat}]")
print(f"Midpoint: [{midpoint_lon}, {midpoint_lat}]")

# Create Earth Engine geometry for the midpoint
midpoint = ee.Geometry.Point([midpoint_lon, midpoint_lat])
coordinates = midpoint.getInfo()['coordinates']

# Find WRS path and row numbers using the midpoint
path_number = int(wrs2_gdf[wrs2_gdf['geometry'].contains(Point(coordinates[0], coordinates[1]))]['PATH'].values[0])
row_number = int(wrs2_gdf[wrs2_gdf['geometry'].contains(Point(coordinates[0], coordinates[1]))]['ROW'].values[0])

# Get the WRS polygon for visualization
wrs2_poly = wrs2_gdf[(wrs2_gdf['PATH']==path_number) & (wrs2_gdf['ROW']==row_number)]['geometry'].values[0]

print(f"WRS Path: {path_number}")
print(f"WRS Row: {row_number}")

Start point: [-67.531973, -13.972296]
End point: [-66.877997, -12.136923]
Midpoint: [-67.204985, -13.0546095]
WRS Path: 1
WRS Row: 69


### Download images and extract graphs

In [11]:
start_x, start_y, end_x, end_y, crs = rb.convert_to_landsat_crs(river_points['start'], river_points['end'], path_number, row_number)

Landsat scene CRS: EPSG:32619
Converted coordinates:
  Start: (2571886.1, -8509859.3)
  End: (2683497.2, -8531926.7)


In [ ]:
# Define river endpoints
point0 = river_points['start'][::-1] # longitude, latitude 
point1 = river_points['end'][::-1] # longitude, latitude 

start_x, start_y, end_x, end_y, crs = rb.convert_to_landsat_crs(
    point0, point1, path_number=path_number, row_number=row_number
)

# Processing parameters
processing_params = {
    'radius': 50,
    'ch_belt_smooth_factor': 1e8,
    'ch_belt_half_width': 2000,
    'remove_smaller_components': False,
    'small_hole_threshold': 64,
    'solidity_filter': False,
    'plot_D_primal': False, 
    'flip_outlier_edges': False,
    'check_edges': False,
    'mndwi_threshold': 0.01
}

# Process a subset of years for demonstration
years_to_process = range(1995, 1996) #range(1984, 2026)

successful_rivers, failed_scenes = rb.River.batch_process_landsat_scenes(
    path_number=path_number,
    row_number=row_number,
    start_x=start_x,
    start_y=start_y,
    end_x=end_x,
    end_y=end_y,
    years=years_to_process,
    max_cloud_cover=5,
    n_scenes_per_year=4,
    clear_rasters=True,
    save_individual=True,
    save_dir='rio_beni_results',
    skip_scenes=[], # skip 'LT05_226079_20071004' for Jurua, 'LT05_232071_20040919' for Mamore
    download_mndwi=True,
    mndwi_dir='/Users/zoltan/Documents/Rivabar/mndwi_data/Beni_test',
    download_false_color=False,
    false_color_dir='/Users/zoltan/Documents/Rivabar/Beni_false_color_images',
    **processing_params
)

print(f"Results: {len(successful_rivers)} successful rivers")

Landsat scene CRS: EPSG:32619
Converted coordinates:
  Start: (658567.8, -1545153.0)
  End: (730930.5, -1342593.8)
💾 Individual rivers will be saved to: rio_beni_results
🗺️  MNDWI rasters will be saved to: /Users/zoltan/Documents/Rivabar/mndwi_data/Beni_test
🛰️  Batch processing Landsat scenes for Path 1, Row 69
Years: 1995-1995, Max cloud cover: 5%
Processing parameters: {'radius': 50, 'ch_belt_smooth_factor': 100000000.0, 'ch_belt_half_width': 2000, 'remove_smaller_components': False, 'small_hole_threshold': 64, 'solidity_filter': False, 'plot_D_primal': False, 'flip_outlier_edges': False, 'check_edges': False, 'mndwi_threshold': 0.01}

--- Processing year 1995 ---
Found 4 scenes for 1995
  Processing scene 1: LT05_001069_19950808
    Date: 1995-08-08, Cloud cover: 0.0%
    ⏭️  Skipping LT05_001069_19950808 (already processed)
  Processing scene 2: LT05_001069_19950605
    Date: 1995-06-05, Cloud cover: 1.0%
    ⏭️  Skipping LT05_001069_19950605 (already processed)
  Processing scene

In [14]:
# Load rivers from pickle files

from rivabar.river import River
rivers = River.load_batch_results('rio_beni_results', min_file_size_kb=1000)
# rivers = River.load_batch_results('rio_mamore_results', min_file_size_kb=1500)
# Get all acquisition dates and create color mapping
dates = [pd.to_datetime(river.acquisition_date) for river in rivers]
# Sort rivers by acquisition date
sorted_indices = sorted(range(len(dates)), key=lambda i: dates[i])
rivers = [rivers[i] for i in sorted_indices]
dates = [dates[i] for i in sorted_indices]
len(rivers)

📂 Found 105 river files in rio_beni_results
🔍 Only loading files larger than 1000 KB
✅ Successfully loaded 103 rivers
⏭️ Skipped 2 files smaller than 1000 KB


103

### Plot deposition and erosion on false color image

In [43]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,15), sharex=True, sharey=True)
chs, bars, erosions, aoi_dates, aoi_centerlines = rb.create_and_plot_bars(rivers, 0, len(rivers)-1, ax1=ax1, ax2=ax2, 
                                                                          depo_cmap='plasma', erosion_cmap='plasma', alpha=1)

100%|██████████| 91/91 [00:45<00:00,  2.00it/s]


In [22]:
ch_lengths = []
ch_widths = []
for river in rivers:
    x1 = river.main_channel_centerline.xy[0]
    y1 = river.main_channel_centerline.xy[1]
    s_dist = rb.compute_s_distance(x1, y1)
    ch_lengths.append(s_dist[-1])
    w = []
    for s,e,d in river.main_path:
        key1 = list(river._D_primal[s][e][d]['half_widths'].keys())[0]
        key2 = list(river._D_primal[s][e][d]['half_widths'].keys())[1]
        w1 = river._D_primal[s][e][d]['half_widths'][key1]
        w2 = river._D_primal[s][e][d]['half_widths'][key2]
        w += list(np.array(w1)+np.array(w2))
    W = 30*np.mean(w)
    ch_widths.append(W)

# Filter out anomalously low channel lengths
# Calculate statistics to identify anomalies
mean_length = np.mean(ch_lengths)
std_length = np.std(ch_lengths)
threshold = mean_length - 0.5 * std_length  # 2 standard deviations below mean

# Create a mask for valid rivers
valid_indices = [i for i, length in enumerate(ch_lengths) if length > threshold]
filtered_rivers = [rivers[i] for i in valid_indices]
filtered_lengths = [ch_lengths[i] for i in valid_indices]

print(f"Original number of rivers: {len(rivers)}")
print(f"Filtered number of rivers: {len(filtered_rivers)}")
print(f"Removed {len(rivers) - len(filtered_rivers)} anomalously short channels")

plt.figure()
plt.plot(ch_lengths, '.-', label='All channels')
plt.plot(valid_indices, filtered_lengths, 'go', label='Valid channels')
plt.axhline(y=threshold, color='r', linestyle='--', label=f'Threshold ({threshold:.2f})')
plt.legend()
plt.title('Channel Lengths with Anomaly Detection')
plt.xlabel('River Index')
plt.ylabel('Channel Length')

Original number of rivers: 103
Filtered number of rivers: 92
Removed 11 anomalously short channels


Text(0, 0.5, 'Channel Length')

In [24]:
rivers = filtered_rivers

In [25]:
results = rb.analyze_river_pairs_filtered(
    rivers, 
    delta_s=100, 
    smoothing_factor=1e6,
    savgol_factor=21,
    min_width_m=280,
    allowed_months=[4,5,6,7,8,9,10],
    min_time_gap_days=270,
    max_time_gap_years=5
)

# Extract your original variables
Lags = results['lags']
costs = results['costs'] 
time_gaps = results['time_gaps']

Filtering rivers by acquisition criteria...
Rivers passing filters: 79 out of 92
  Month filter ([4, 5, 6, 7, 8, 9, 10]): Included rivers from allowed months
  Width filter (≥280m): Included rivers with sufficient width

Analyzing river pairs...


100%|██████████| 79/79 [04:41<00:00,  3.57s/it]


Analysis Summary:
  Total possible pairs: 3081
  Valid pairs analyzed: 688
  Success rate: 22.3%
  Time gap range: 272 to 1824 days (0.7 to 5.0 years)
  Cost range: 3398.208 to 3970.842
  Filters applied:
    - Min time gap: 270 days (0.7 years)
    - Max time gap: 5 years
    - Allowed months: [4, 5, 6, 7, 8, 9, 10]
    - Min width: 280m


In [57]:
import pickle
import gzip

# Create a new dict without the problematic River objects
results_to_save = {
    'lags': results['lags'],
    'costs': results['costs'],
    'time_gaps': results['time_gaps'],
    'pair_info': results['pair_info'],
    'widths': results['widths'],
    'curvatures': results['curvatures'],
    'migration_distances': results['migration_distances'],
    'along_channel_distances': results['along_channel_distances'],
    'centerline_coords': results['centerline_coords'],
    'n_total_pairs': results['n_total_pairs'],
    'n_valid_pairs': results['n_valid_pairs']
    # Skip 'filtered_rivers' and 'filter_info' which contain River objects
}

# Save the cleaned results
with gzip.open('rio_beni_pair_results.pkl.gz', 'wb') as f:
    pickle.dump(results_to_save, f)

In [27]:
from scipy import stats
r_squareds = []
p_values = []
p90_dists = []
mean_lengths = []

from tqdm import trange

for pair_idx in trange(len(results['pair_info'])):
    # Get data for a specific river pair (using the first pair as an example)
    # pair_idx = 125  # Change this to select a different river pair
    pair_info = results['pair_info'][pair_idx]
    distances = results['migration_distances'][pair_idx]
    curvature1 = results['curvatures'][pair_idx]
    s1 = results['along_channel_distances'][pair_idx]

    # Extract the river indices for this pair
    river1_idx = pair_info['river1_index']
    river2_idx = pair_info['river2_index']

    # Get the mean width to use as W
    W = pair_info['width1_m']

    # Get the lag for this specific pair
    pair_lag = np.mean(results['lags'][pair_idx])//100
    pair_lag = pair_lag.astype(int)

    # Eliminate outliers using z-score method
    if pair_lag < 0:
        x = -curvature1[:pair_lag]*W
        y = distances[-1*pair_lag:]/W
    elif pair_lag ==0:
        x = -curvature1*W
        y = distances/W
    else:
        x = -curvature1[:-pair_lag]*W
        y = distances[1*pair_lag:]/W

    # Calculate z-scores
    x_zscore = np.abs(stats.zscore(x))
    y_zscore = np.abs(stats.zscore(y))

    # Filter out outliers (typically z-score > 3 indicates outliers)
    mask = (x_zscore < 3) & (y_zscore < 3)
    x_filtered = x[mask]
    y_filtered = y[mask]

    # Add linear regression on filtered data
    slope, intercept, r_value, p_value, std_err = stats.linregress(x_filtered, y_filtered)

    r_squared = r_value**2
    r_squareds.append(r_squared)
    p_values.append(p_value)
    p90_dists.append(np.percentile(np.abs(distances/pair_info['time_gap_days']), 90))
    mean_lengths.append(0.5*(results['pair_info'][pair_idx]['s1'][-1] + results['pair_info'][pair_idx]['s2'][-1]))

100%|██████████| 688/688 [00:00<00:00, 1663.19it/s]


In [31]:
mean_lags = []
midpoint_dates = []

# Calculate mean lag for each river pair
for i, lag in enumerate(Lags):
    mean_lags.append(np.mean(lag))
    
    # Calculate the midpoint date for each river pair
    # time_gaps contains the time difference between river pairs in days
    # We need to extract the corresponding dates from the results
    pair_indices = results['pair_info'][i]['river1_index'], results['pair_info'][i]['river2_index']
    date1 = pd.to_datetime(rivers[pair_indices[0]].acquisition_date)
    date2 = pd.to_datetime(rivers[pair_indices[1]].acquisition_date)
    midpoint_date = date1 + (date2 - date1) / 2
    midpoint_dates.append(midpoint_date)

# Convert to numpy arrays for easier plotting
mean_lags = np.array(mean_lags)
midpoint_dates = np.array(midpoint_dates)

# Sort by midpoint date
sort_idx = np.argsort(midpoint_dates)
midpoint_dates_sorted = midpoint_dates[sort_idx]
mean_lags_sorted = mean_lags[sort_idx]

plt.figure()
plt.plot(midpoint_dates_sorted, mean_lags_sorted, 'o')
plt.xlabel('Time (midpoint of river pair)')
plt.ylabel('Mean Lag')
plt.title('Mean Lag vs Time')
plt.gcf().autofmt_xdate()  # Rotate date labels for better readability

In [45]:
mean_widths = []
for i in range(len(results['pair_info'])):
    width = 0.5*(results['pair_info'][i]['width1_m'] + results['pair_info'][i]['width2_m'])
    mean_widths.append(np.mean(width))
mean_widths = np.array(mean_widths)

norm_lag = np.abs(mean_lags[r_squareds**0.5>0.55])/mean_widths[r_squareds**0.5>0.55]
norm_dist = p90_dists[r_squareds**0.5>0.55]*365/mean_widths[r_squareds**0.5>0.55]
df_lag_mr = pd.DataFrame({'norm_lag': norm_lag, 'norm_dist': norm_dist})
plt.figure()
plt.scatter(df_lag_mr['norm_lag'], df_lag_mr['norm_dist'])
plt.xlabel('Normalized lag (lag / channel width)')
plt.ylabel('Normalized P90 migration rate (channel widths / year)');

In [47]:
p90_dists_sorted = p90_dists[sort_idx]
r_squareds_sorted = r_squareds[sort_idx]
mean_widths_sorted = mean_widths[sort_idx]

plt.figure()
plt.scatter(midpoint_dates_sorted[r_squareds_sorted**0.5>0.4], p90_dists_sorted[r_squareds_sorted**0.5>0.4]*365)
plt.xlabel('Time (midpoint of river pair)')
plt.ylabel('90th percentile of migration rate (m/year)')
plt.title('Migration rate vs time, Rio Beni');

In [49]:
import seaborn as sns

# Get data for a specific river pair (using the first pair as an example)
pair_idx = 600  # Change this to select a different river pair
pair_info = results['pair_info'][pair_idx]
distances = results['migration_distances'][pair_idx]
curvature1 = results['curvatures'][pair_idx]
s1 = results['along_channel_distances'][pair_idx]

# Extract the river indices for this pair
river1_idx = pair_info['river1_index']
river2_idx = pair_info['river2_index']

# Get the mean width to use as W
W = pair_info['width1_m']

# Create the visualization
fig, axes = plt.subplots(2, 1, figsize=(25,6), sharex=True)
plt.tight_layout()
axes[0].fill_between(s1, 0, -curvature1*W)
axes[1].fill_between(s1, 0, distances/W, facecolor='green')
axes[0].set_xlim(0, s1[-1])
percentile = 1
max_abs_value = max(abs(np.percentile(distances/W, percentile)), abs(np.percentile(distances/W, 100-percentile)))
axes[1].set_ylim(-max_abs_value, max_abs_value)
axes[0].set_ylabel('Curvature')
axes[1].set_ylabel('Migration Distance')
axes[1].set_xlabel('Distance along channel')
axes[0].set_title(f'River Pair {pair_idx}')

plt.figure()
# Get the lag for this specific pair
pair_lag = np.mean(results['lags'][pair_idx])//100
pair_lag = pair_lag.astype(int)

# Create KDE plot
sns.kdeplot(x=-curvature1[:pair_lag]*W, y=distances[-1*pair_lag:]/W)

from scipy import stats

# Eliminate outliers using z-score method
x = -curvature1[:pair_lag]*W
y = distances[-1*pair_lag:]/W

# Calculate z-scores
x_zscore = np.abs(stats.zscore(x))
y_zscore = np.abs(stats.zscore(y))

# Filter out outliers (typically z-score > 3 indicates outliers)
mask = (x_zscore < 3) & (y_zscore < 3)
x_filtered = x[mask]
y_filtered = y[mask]

# Add linear regression on filtered data
slope, intercept, r_value, p_value, std_err = stats.linregress(x_filtered, y_filtered)
r_squared = r_value**2

# Plot regression line
x_line = np.linspace(min(x_filtered), max(x_filtered), 100)
y_line = slope * x_line + intercept
plt.plot(x_line, y_line, 'r-')

# Add text with R-squared value
plt.text(0.05, 0.95, f'R² = {r_squared:.3f}', transform=plt.gca().transAxes, 
         bbox=dict(facecolor='white', alpha=0.7));

In [ ]:
from librosa.sequence import dtw

mr = distances/W
curv = -curvature1*W

sm = np.zeros((len(s1),len(s1))) # similarity matrix
for i in range(0,len(s1)):
    sm[i,:] = (np.abs(mr - curv[i]))**0.07
D, wp = dtw(C = sm) # dynamic time warping
p1 = wp[:,0] # correlation indices for first curve
q1 = wp[:,1] # correlation indices for second curve
p1 = np.array(p1)
q1 = np.array(q1)

plt.figure()
plt.plot(s1, curv, '.-')
plt.plot(s1, mr - 1, '.-')
lags = []
for i in range(0, len(p1), 1):
    plt.plot([s1[p1[i]], s1[q1[i]]], [curv[p1[i]], mr[q1[i]] - 1], 'k-', linewidth=0.5)
    lags.append(s1[p1[i]] - s1[q1[i]])
plt.figure()
plt.plot(lags);